In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

base_dir = 'c:/Users/deepa/Documents/Deeplearning CA/deep-learning-techniques-comparison-rnn-tcn'
preprocessed_dir = os.path.join(base_dir, 'preprocessed_data')

In [ ]:
with open(os.path.join(preprocessed_dir, 'bike_sharing.pkl'), 'rb') as f:
    bike_data = pickle.load(f)

X_train_seq = torch.FloatTensor(bike_data['X_train_seq']).to(device)
X_val_seq = torch.FloatTensor(bike_data['X_val_seq']).to(device)
X_test_seq = torch.FloatTensor(bike_data['X_test_seq']).to(device)

y_train_seq = torch.FloatTensor(bike_data['y_train_seq']).unsqueeze(1).to(device)
y_val_seq = torch.FloatTensor(bike_data['y_val_seq']).unsqueeze(1).to(device)
y_test_seq = torch.FloatTensor(bike_data['y_test_seq']).unsqueeze(1).to(device)

print(f"X_train shape: {X_train_seq.shape}")
print(f"X_val shape: {X_val_seq.shape}")
print(f"X_test shape: {X_test_seq.shape}")
print(f"y_train shape: {y_train_seq.shape}")
print(f"Sequence length: {X_train_seq.shape[1]}, Features: {X_train_seq.shape[2]}")
print(f"Device: {device}")

In [ ]:
class TCN(nn.Module):
    def __init__(self, input_size, num_filters, kernel_size):
        super(TCN, self).__init__()
        self.conv1 = nn.Conv1d(input_size, num_filters, kernel_size, padding='same')
        self.dropout1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv1d(num_filters, num_filters, kernel_size, padding='same')
        self.dropout2 = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(num_filters * 10, 64)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

num_filters = 64
kernel_size = 3
epochs = 50
batch_size = 32
learning_rate = 0.001

input_size = X_train_seq.shape[2]
model = TCN(input_size, num_filters, kernel_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(model)

In [ ]:
train_dataset = TensorDataset(X_train_seq, y_train_seq)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_seq, y_val_seq)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
train_maes = []
val_maes = []

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_mae = 0.0
    
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_mae += torch.mean(torch.abs(outputs - y_batch)).item()
    
    train_losses.append(train_loss / len(train_loader))
    train_maes.append(train_mae / len(train_loader))
    
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            val_loss += loss.item()
            val_mae += torch.mean(torch.abs(outputs - y_batch)).item()
    
    val_losses.append(val_loss / len(val_loader))
    val_maes.append(val_mae / len(val_loader))
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val MAE: {val_maes[-1]:.4f}')

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_seq).cpu().numpy().flatten()
    y_test_np = y_test_seq.cpu().numpy().flatten()

mse = mean_squared_error(y_test_np, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)

print(f"\n{'='*50}")
print(f"PyTorch TCN - Bike Sharing Regression Results")
print(f"{'='*50}")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R²:   {r2:.4f}")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_maes, label='Train MAE')
plt.plot(val_maes, label='Val MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(y_test_np[:100], label='Actual', marker='o')
plt.plot(y_pred[:100], label='Predicted', marker='s')
plt.title('Actual vs Predicted (First 100 samples)')
plt.xlabel('Sample')
plt.ylabel('Bike Count')
plt.legend()
plt.show()